# Forecasting the monthly sales of French champagne

In [8]:
import scipy
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn as skl
import statsmodels as stmd
from math import sqrt

series = pd.read_csv('monthly-champagne-sales.csv', header=0)
dataset, validation = series[0:-12], series[-12:] #test set is the final year
dataset.to_csv('dataset.csv')
validation.to_csv('validation.csv')

# load data
series = pd.read_csv('dataset.csv')

# preparing data
X = series.values
X = X.astype('float32')
train_size = int(len(X) * 0.5)
train, test = X[0:train_size], X[train_size:]

# walk-forward validation
history = [x for x in train]
predictions = list()
for i in range(len(test)):
    # predict
    yhat = history[-1]
    predictions.append(yhat)
    # observation
    obs = test[i]
    history.append(obs)
    print('>Predicted=%.3f, Expected=%.3f' % (yhat, obs))

# report performance
rmse = sqrt(mean_squared_error(test, predictions))
print('RMSE: %.3f' % rmse)

ValueError: could not convert string to float: '1971-10'